---
title: 堆糖图片多线程抓取
date: 2019-08-26
categories: [爬虫开发, 项目]
mathjax: false
---

In [76]:
import requests
import urllib.parse
import threading

#多线程锁
thread_lock = threading.BoundedSemaphore(value = 10)

def get_page(url):
    #GET之后用utf-8解析
    page = requests.get(url).content.decode("utf-8")
    return page

def findall_in_page(page, start_part, end_part):
    all_strings = []
    end = 0
    while page.find(start_part,end) != -1:
        start = page.find(start_part, end)+len(start_part)
        end = page.find(end_part, start)
        string = page[start: end]
        all_strings.append(string)
    return all_strings

def pic_urls_find_part(pages):
    pic_pics = []
    for page in pages:
        urls = findall_in_page(page, '"path":"', '"')
        pic_pics.extend(urls)
    return pic_pics
    
def pages_usr(label):
    pages = []
    url = 'https://www.duitang.com/napi/blog/list/by_search/?kw={}&start={}&limit=1000'
    #将中文转成ASCII码
    label = urllib.parse.quote(label)
    #观察response可发现max=3600
    for index in range(0,3600,100):
        u = url.format(label,index)
        page = get_page(u)
        pages.append(page)
        return pages
        
def download_pic(url, name):
    req = requests.get(url)
    path = 'test/杨幂'+str(name)+".jpg"
    with open(path, 'wb+') as file:
        file.write(req.content)
    #将锁打开避免堵塞
    thread_lock.release()
        
def main(label):
    pages = pages_usr(label)
    pic_urls = pic_urls_find_part(pages)
    numbers = 0
    for url in pic_urls:
        numbers += 1
        print("Now Downloading: {}".format(numbers))
        
        #锁上，并开始执行
        thread_lock.acquire()
        t = threading.Thread(target=download_pic, args=(url, numbers))
        t.start()
        
main('杨幂')

Now Downloading: 1
Now Downloading: 2
Now Downloading: 3
Now Downloading: 4
Now Downloading: 5
Now Downloading: 6
Now Downloading: 7
Now Downloading: 8
Now Downloading: 9
Now Downloading: 10
Now Downloading: 11
Now Downloading: 12
Now Downloading: 13
Now Downloading: 14
Now Downloading: 15
Now Downloading: 16
Now Downloading: 17
Now Downloading: 18
Now Downloading: 19
Now Downloading: 20
Now Downloading: 21
Now Downloading: 22
Now Downloading: 23
Now Downloading: 24
Now Downloading: 25
Now Downloading: 26
Now Downloading: 27
Now Downloading: 28
Now Downloading: 29
Now Downloading: 30
Now Downloading: 31
Now Downloading: 32
Now Downloading: 33
Now Downloading: 34
Now Downloading: 35
Now Downloading: 36
Now Downloading: 37
Now Downloading: 38
Now Downloading: 39


KeyboardInterrupt: 

> 参考：

1. [从零起步 系统入门Python爬虫工程师](https://coding.imooc.com/class/chapter/325.html)